# Preprocessing & Flattening

Due to the size of these datasets as well as the JSON fields contained within them we need to seperate the values and flatten the fields as well as throw out any non-changing data. We first work with the training data strip the columns we don't need and release the memory back to use for the test data. It's recommended to shutdown the kernel after to release the memory back to the OS.

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import gc

def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

## Training Data
### Import & Flatten

In [2]:
%%time
train_df = load_df("./input/train_v2.csv")

Loaded train_v2.csv. Shape: (1708337, 60)
Wall time: 13min 41s


### Find Non-Entropy fields

In [3]:
%%time 
const_cols = [c for c in train_df.columns if train_df[c].nunique(dropna=False)==1 ]

Wall time: 4min 54s


### Remove Non-entropy fields

In [4]:
%%time
train_df.drop(const_cols, axis=1, inplace=True)

Wall time: 4min 52s


### Write CSV back out

In [5]:
%%time
train_df.to_csv("train_flattened.csv", index=False)

Wall time: 11min 18s


### Delete pandas dataframe and call garbage collection

In [6]:
%%time
del train_df
gc.collect()

Wall time: 1min 21s


## Test Data
### Import & Flatten

In [7]:
%%time
test_df = load_df("./input/test_v2.csv")

Loaded test_v2.csv. Shape: (401589, 59)
Wall time: 1min 40s


### Remove Non-entropy fields

In [8]:
%%time
test_df.drop(const_cols, axis=1, inplace=True)

Wall time: 1.49 s


### Write CSV back out

In [9]:
%%time
test_df.to_csv("test_flattened.csv", index=False)

Wall time: 2min 25s


### Delete pandas dataframe and call garbage collection

In [10]:
%%time
del test_df
gc.collect()

Wall time: 16.1 s


## Resource Usage!

## Before & After
### Memory
<div width=100%>
    <img src="./images/memory_before.png" width=45%/>
    <img src="./images/memory_after.png" width=45%/>
</div>

### CPU
<div width=100%>
    <img src="./images/cpu_before.png" width=45%/>
    <img src="./images/cpu_after.png" width=45%/>
</div>